In [1]:
import pandas as pd
from gensim import models
import jieba
import codecs

In [2]:
def cut_word(sentence):
    return [' '.join([w for w in jieba.lcut(sentence) if w not in stop_words])]

In [18]:
cut_word('银行房屋抵押贷款，从申请到放款一般至少需要十五天。一、银行贷款流程：1.先提交一份个人贷款申')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Zn\AppData\Local\Temp\jieba.cache
Loading model cost 0.523 seconds.
Prefix dict has been built succesfully.


['银行 房屋 抵押 贷款 申请 放款 至少 十五天 银行贷款 流程 先 提交 一份 贷款 申']

In [22]:
df = pd.read_csv(r'G:\BaiduNetdiskDownload/financezhidao_filter.csv').fillna('')

In [23]:
df = df.fillna('')
df.sample(5)

,title,question,reply,is_best
451840,怎么选择理财产品？,,保本为第一要务,0
499598,网上怎么查中*人民银行征信记录,,各种都可以查，详情点击进入各种都可以查，详情点击进入各种都可以查，详情点击进入首先，要明白自...,0
305483,房产抵押贷款的放款时间是多久,,银行房屋抵押贷款，从申请到放款一般至少需要十五天。一、银行贷款流程：1.先提交一份个人贷款申...,0
571433,取款密码错了三次然后还可以再网银上预约取款么？,,不可以的取卡密码错误超限你要去银行柜台办理密码重置才行携带身份证去银行柜台办理即可密码锁死不...,1
298408,身份证丢失办银行卡需要什么证件,,如办理招行一卡通，需提供身份证原件的，若证件丢失，建议您尽快挂失证件补办后再办理一卡通。20...,1


In [24]:
corpus = df.title.tolist()
corpus.extend(df.reply.tolist())
corpus.extend(df.question.tolist())

In [16]:
stop_words = ''.join(codecs.open('./input/stop_words.txt', encoding='utf-8').readlines()).split('\n')
# txt = [cut_word(s) for s in corpus]

In [84]:
df = pd.read_csv('finance_cut_word.csv').fillna('')

In [85]:
df.columns = ['sentence']

In [86]:
df.sample(10)

,sentence
1221629,信用卡 取现 取现 额度 超过 信用卡 可用 额度 取 现有 手续费 银行 信用卡 收取 手...
1289634,招行 一卡通 请 致电 #### 选 人工 服务 办理 口头 挂失 忘记 卡号 提供 开户 ...
1017858,代 收 个点 费用 按天来 收取 刷 1.25 机子 收 个点 刷 0.38 收 个点
561213,招商银行 经典 白金 信用卡 第一年 免年费
809989,招商银行 兑换 请 打开 查询 外汇 实时 汇率 汇率 实时 变动 实际汇率 实际操作 时 ...
1597070,二手房 房款 30 万 按揭 贷款 一个月 看中 一套 二手房 房价 29 万 按揭 首付 ...
71426,投资 理财 心态
494718,网银 跨行 转账 实时 账 需 多久 账
864831,2345 贷款 王富 信 现金 贷 额度
933561,持有 我行 信用卡 申请 一张 额度 共享 信用卡 统一 账户 管理 名下 有多张 信用卡 ...


In [105]:
words = df.sentence.tolist()

In [106]:
[i for i in words[0].split(' ')]

['壹方', '水榭', '用深', '公积金', '贷款', '壹方', '水榭', '套房']

In [94]:
words = [[i for i in w.split(' ')] for w in words]

In [99]:
words

{'水榭': <gensim.models.keyedvectors.Vocab at 0x1ad171f3828>,
 '用深': <gensim.models.keyedvectors.Vocab at 0x1ad171f3978>,
 '公积金': <gensim.models.keyedvectors.Vocab at 0x1ad171f39b0>,
 '贷款': <gensim.models.keyedvectors.Vocab at 0x1ad171f3898>,
 '套房': <gensim.models.keyedvectors.Vocab at 0x1ad171f3860>,
 '多久': <gensim.models.keyedvectors.Vocab at 0x1ad171f3908>,
 '放款': <gensim.models.keyedvectors.Vocab at 0x1ad171f39e8>,
 '迟迟': <gensim.models.keyedvectors.Vocab at 0x1ad171f3a20>,
 '快速': <gensim.models.keyedvectors.Vocab at 0x1ad171f3a58>,
 '借款': <gensim.models.keyedvectors.Vocab at 0x1ad171f3a90>,
 '逾期': <gensim.models.keyedvectors.Vocab at 0x1ad171f3ac8>,
 '转入': <gensim.models.keyedvectors.Vocab at 0x1ad171f3b00>,
 '账': <gensim.models.keyedvectors.Vocab at 0x1ad171f3b38>,
 '查询': <gensim.models.keyedvectors.Vocab at 0x1ad171f3b70>,
 '进度': <gensim.models.keyedvectors.Vocab at 0x1ad171f3ba8>,
 '悦': <gensim.models.keyedvectors.Vocab at 0x1ad171f3be0>,
 '借钱': <gensim.models.keyedvectors.Vocab 

In [95]:
model = models.Word2Vec(words, size=50, window=10, min_count=5, hs=1, workers=8)

In [96]:
words = model.wv.vocab
ci = [] #得到所有的词
for word in words:
    ci.append(word)

In [100]:
outerdict = {}
for index, w in  enumerate(ci):
    innerdict = dict()
    f = model.wv.most_similar(positive=[w], negative=[], topn=8)
    for word in f:
        innerdict[word[0]] = word[1]
    outerdict[w] = innerdict
    if index % 2000 == 0:
        print('-----' + str(index) + 'rows done---------')

-----0rows done---------
-----2000rows done---------
-----4000rows done---------
-----6000rows done---------
-----8000rows done---------
-----10000rows done---------
-----12000rows done---------
-----14000rows done---------
-----16000rows done---------
-----18000rows done---------
-----20000rows done---------
-----22000rows done---------
-----24000rows done---------
-----26000rows done---------
-----28000rows done---------
-----30000rows done---------
-----32000rows done---------
-----34000rows done---------
-----36000rows done---------
-----38000rows done---------
-----40000rows done---------
-----42000rows done---------


In [102]:
import json
#将字典存入json文件
with open('./model/finance.json', 'w', encoding = 'utf-8') as f:
    json.dump(outerdict, f, ensure_ascii=False, indent=4)

In [103]:
outerdict

{'水榭': {'111': 0.6719089150428772,
  '人民广场': 0.6808258891105652,
  '坂田': 0.6806032657623291,
  '开发区': 0.6812878847122192,
  '惠阳': 0.673157811164856,
  '滨河': 0.6742401123046875,
  '胶州': 0.7056731581687927,
  '高新': 0.6867814064025879},
 '用深': {'公交公司': 0.7688904404640198,
  '固安': 0.7548428773880005,
  '实习医生': 0.7580220103263855,
  '沁': 0.729188859462738,
  '焦作': 0.7559258937835693,
  '用北': 0.8159042000770569,
  '贷转': 0.755831241607666,
  '铜仁': 0.8255820870399475},
 '公积金': {'5.04%': 0.5437568426132202,
  '中去': 0.5386804938316345,
  '买商': 0.5816134214401245,
  '公积': 0.5577384233474731,
  '商转公': 0.5454143285751343,
  '积金': 0.5945847034454346,
  '职工': 0.6205554604530334,
  '购建': 0.5267482995986938},
 '贷款': {'买房子': 0.49620893597602844,
  '代款': 0.47633096575737,
  '房屋贷款': 0.5818988084793091,
  '殴元': 0.49490341544151306,
  '贷': 0.5019299387931824,
  '车贷': 0.5634583234786987,
  '车辆': 0.47959062457084656,
  '这要': 0.5374853014945984},
 '套房': {'一套': 0.5526711940765381,
  '买房子': 0.567341148853302,
  